In [2]:
import os
import pickle
import time
from src.retriever.bm25_retriever import BM25
from src.retriever.tfidf_retriever import TFIDF
from src.retriever.faiss_retriever import FaissRetriever
from src.retriever.milvus_retriever import MilvusRetriever 
from src.client.llm_local_client import request_chat
from src.client.llm_hyde_client import request_hyde
from src.reranker.bge_m3_reranker import BGEM3ReRanker 
from src.constant import bge_reranker_tuned_model_path
from src.utils import merge_docs, post_processing

# warmstart
bm25_retriever = BM25(docs=None, retrieve=True)
milvus_retriever = MilvusRetriever(docs=None, retrieve=True) 
bge_m3_reranker = BGEM3ReRanker(model_path=bge_reranker_tuned_model_path)
milvus_retriever.retrieve_topk("这是一条测试数据", topk=3)

Successfully connected to MongoDB


2025-09-28 15:18:29.573644: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1759043909.617263    1198 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1759043909.628728    1198 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1759043909.693535    1198 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759043909.693576    1198 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1759043909.693585    1198 computation_placer.cc:177] computation placer alr

Successfully connected to MongoDB


You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[Document(metadata={'unique_id': '2778d2639947194feb0d926679b37845', 'source': '/root/autodl-tmp/RAG/data/Tesla_Manual.pdf', 'page': 245, 'images_info': [], 'parent_id': '436269f79cddae0c5e123a06cd364206'}, page_content='### 质量控制\n交付Model 3时，您可能会注意到里程表上有一些公里数，这是为确保Model 3的质量而进行全面测试的结果。测试过程包括生产期间和之后的全面检查，最终检查由Tesla完成，包括由技术人员进行的道路试验。'),
 Document(metadata={'unique_id': '3493ca35d41ddb7627c1e8e0cf6bcc76', 'source': '/root/autodl-tmp/RAG/data/Tesla_Manual.pdf', 'page': 248, 'images_info': []}, page_content='### 认证合格性\n（文档中未提及具体内容，仅给出标题）'),
 Document(metadata={'unique_id': '436269f79cddae0c5e123a06cd364206', 'source': '/root/autodl-tmp/RAG/data/Tesla_Manual.pdf', 'page': 245, 'images_info': []}, page_content='### 质量控制\n交付Model 3时，您可能会注意到里程表上有一些公里数，这是为确保Model 3的质量而进行全面测试的结果。测试过程包括生产期间和之后的全面检查，最终检查由Tesla完成，包括由技术人员进行的道路试验。\n\n')]

In [5]:
query = "请给我介绍一下特斯拉model3，有没有离车自动上锁功能，怎么使用的呢"

In [22]:
# 检索
# BM25召回
t1 = time.time()
bm25_docs = bm25_retriever.retrieve_topk(query, topk=3)
print("BM25召回样例:")
print(bm25_docs)
print("="*100)
t2 = time.time()
print(t2-t1)

BM25召回样例:
[Document(metadata={'unique_id': 'aad23c04e3c3af29c0edce4645ab443a', 'source': '/root/autodl-tmp/RAG/data/Tesla_Manual.pdf', 'page': 28, 'images_info': [{'page': 28, 'image_path': '/root/autodl-tmp/RAG/data/saved_images/page28_img1.jpeg', 'title': '儿童安全座椅'}, {'page': 28, 'image_path': '/root/autodl-tmp/RAG/data/saved_images/page28_img2.jpeg', 'title': ''}, {'page': 28, 'image_path': '/root/autodl-tmp/RAG/data/saved_images/page28_img3.jpeg', 'title': '配对'}], 'parent_id': 'b01e5a84030163a6f22a6edaf025975a'}, page_content='### 智能儿童座椅应用程序\n- 注：在使用智能儿童座椅应用程序前，Tesla建议先启用离车后自动上锁（请查阅离车后自动上锁页码7）。Tesla还建议您不要启用住宅除外，这样车辆就会始终锁定。'), Document(metadata={'unique_id': '4370ec3210e8a67d40a2b69be096fc33', 'source': '/root/autodl-tmp/RAG/data/Tesla_Manual.pdf', 'page': 10, 'images_info': [], 'parent_id': '17dd4e1f8ed1c04ea9b08b3313ac2ad9'}, page_content='- 某个车门或行李箱未完全关闭。\n- 手机钥匙的蓝牙设置已关闭。\n- 如果Model 3在您下车并关上所有车门后几分钟内探测到经认证的钥匙，离车后自动上锁功能便会失效，当您离车时车门不会上锁。在这种情况下，您必须手动锁定车辆，直至下次驾驶。\n- 驾驶员不使用驾驶位车门下车。\n注：

In [23]:
# BGE-M3稠密+稀疏召回+RRF初排
milvus_docs = milvus_retriever.retrieve_topk(query, topk=3)
print("BGE-M3召回样例:")
print(milvus_docs)
print("="*100)
t3 = time.time()

2025-09-28 15:32:55,567 [ERROR][handler]: RPC error: [hybrid_search], <MilvusException: (code=<bound method _InactiveRpcError.code of <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmp9fd6640q_milvus.db.sock: connect: Connection refused (111)"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-09-28T15:32:55.56672327+08:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmp9fd6640q_milvus.db.sock: connect: Connection refused (111)"}"
>>, message=[hybrid_search] Retry run out of 75 retry times, message=failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmp9fd6640q_milvus.db.sock: connect: Connection refused (111))>, <Time:{'RPC start': '2025-09-28 15:29:24.809763', 'RPC error': '2025-09-28 15:32:55.566998'}> (decorators.py:140)


MilvusException: <MilvusException: (code=<bound method _InactiveRpcError.code of <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmp9fd6640q_milvus.db.sock: connect: Connection refused (111)"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-09-28T15:32:55.56672327+08:00", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmp9fd6640q_milvus.db.sock: connect: Connection refused (111)"}"
>>, message=[hybrid_search] Retry run out of 75 retry times, message=failed to connect to all addresses; last error: UNKNOWN: unix:/tmp/tmp9fd6640q_milvus.db.sock: connect: Connection refused (111))>

In [ ]:
# 去重
merged_docs = merge_docs(bm25_docs, milvus_docs)
print(merged_docs)
print("="*100)


# 精排 
ranked_docs = bge_m3_reranker.rank(query, merged_docs, topk=5)
print(ranked_docs)
print("="*100)


# 答案
context = "\n".join(["【" + str(idx+1) + "】" + doc.page_content for idx, doc in enumerate(ranked_docs)])
res_handler = request_chat(query, context, stream=True)
response = ""
for r in res_handler:
    uttr = r.choices[0].delta.content
    response += uttr 
    print(uttr, end='')
print("\n" + "="*100)

# 后处理
answer = post_processing(response, ranked_docs)
print("\n答案—>", answer)